Q1 

path number: 10000 option price: 6.1903 standard error: 0.0826

path number: 40000 option price: 6.0974 standard error: 0.0405

In [1]:
import numpy as np
from scipy.stats import norm

# 参数初始化
sigma1 = 1
beta0 = 0.5
beta1 = 0.3
beta2 = 0.5

m = 10
n = 10000

# 生成标准正态分布的随机数矩阵
rnorm = np.random.normal(0, 1, (n, m))

# 初始化数组S
S = np.zeros(n)

# 主要计算循环
for x in range(n):
    sigmai = np.zeros(m)
    sigmai[0] = sigma1
    Xi = np.zeros(m)
    for y in range(m):
        Xi[y] = sigmai[y] * rnorm[x, y]
        if not y == m - 1:
            sigmai[y + 1] = np.sqrt(beta0 + beta1 * Xi[y] ** 2 + beta2 * sigmai[y] ** 2)
    S[x] = sum(Xi)

# 计算5%和1%分位数的VaR
VaR_5 = -np.sort(S)[int(n * 0.05)]
VaR_1 = -np.sort(S)[int(n * 0.01)]

# 计算置信区间的上下界
k_5_min = int(n * 0.05 - np.sqrt(n * 0.05 * 0.95) * norm.ppf(0.975))
k_5_max = int(n * 0.05 + np.sqrt(n * 0.05 * 0.95) * norm.ppf(0.975))
k_1_min = int(n * 0.01 - np.sqrt(n * 0.01 * 0.99) * norm.ppf(0.975))
k_1_max = int(n * 0.01 + np.sqrt(n * 0.01 * 0.99) * norm.ppf(0.975))

# 输出相关结果
print(
    VaR_5,
    -np.sort(S)[k_5_max],
    -np.sort(S)[k_5_min],
    VaR_1,
    -np.sort(S)[k_1_max],
    -np.sort(S)[k_1_min],
)


6.909708776342058 6.730885828264311 7.118059570833675 10.840933665874976 10.394397703449686 11.281275508238446


Q2

p = 0.05 :  VaR = 6.9698, 95% CI = [6.7703, 7.2018]

p = 0.01 :  VaR = 11.2231, 95% CI = [10.6132, 11.6618]

In [2]:
import numpy as np
from scipy.stats import norm

sigma1 = 1
beta0 = 0.5
beta1 = 0.3
beta2 = 0.5

m = 10
n = 10000

rnorm = np.random.normal(0, 1, (n, m))

S = np.zeros(n)

for x in range(n):
    sigmai = np.zeros(m)
    sigmai[0] = sigma1
    Xi = np.zeros(m)
    for y in range(m):
        Xi[y] = sigmai[y] * rnorm[x, y]
        if not y == m - 1:
            sigmai[y + 1] = np.sqrt(beta0 + beta1 * Xi[y] ** 2 + beta2 * sigmai[y] ** 2)
    S[x] = sum(Xi)

VaR_5 = -np.sort(S)[int(n * 0.05)]
VaR_1 = -np.sort(S)[int(n * 0.01)]

k_5_min = int(n * 0.05 - np.sqrt(n * 0.05 * 0.95) * norm.ppf(0.975))
k_5_max = int(n * 0.05 + np.sqrt(n * 0.05 * 0.95) * norm.ppf(0.975))
k_1_min = int(n * 0.01 - np.sqrt(n * 0.01 * 0.99) * norm.ppf(0.975))
k_1_max = int(n * 0.01 + np.sqrt(n * 0.01 * 0.99) * norm.ppf(0.975))

print(
    f"p = 0.05 :  VaR = {VaR_5:.4f}, 95% CI = [{-np.sort(S)[k_5_max]:.4f}, {-np.sort(S)[k_5_min]:.4f}]"
)
print(
    f"p = 0.01 :  VaR = {VaR_1:.4f}, 95% CI = [{-np.sort(S)[k_1_max]:.4f}, {-np.sort(S)[k_1_min]:.4f}]"
)

p = 0.05 :  VaR = 7.0593, 95% CI = [6.8338, 7.3623]
p = 0.01 :  VaR = 11.1299, 95% CI = [10.7532, 11.6354]


Q3

Plain Monte Carlo: Price = 4.9866, SE = 0.0672

Antithetic Sampling: Price = 5.0058, SE = 0.0343

Control Variate: Price = 5.0044, SE = 0.0283

In [3]:
import numpy as np
from scipy.stats import norm


def black_scholes_call(S0, K, T, r, sigma):
    """
    Calculate the Black-Scholes price of a European call option.
    """
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price


def simulate_paths_plain(S0, K, r, sigma, T, m, n, seed=None):
    """
    Plain Monte Carlo simulation: Generate n paths, each with m time steps.
    Calculate the payoff of the lookback call option = (max S - K)^+ for each path.
    """
    if seed is not None:
        np.random.seed(seed)
    dt = T / m
    # Generate n×m standard normal random numbers
    Z = np.random.randn(n, m)
    increments = (r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z
    # Cumulative sum to get log(S); initial value is log(S0)
    logS = np.log(S0) + np.cumsum(increments, axis=1)
    S = np.exp(logS)
    S_max = np.max(S, axis=1)
    payoff = np.maximum(S_max - K, 0)
    return payoff


def plain_monte_carlo(S0, K, r, sigma, T, m, n, seed=None):
    payoff = simulate_paths_plain(S0, K, r, sigma, T, m, n, seed)
    discounted_payoff = np.exp(-r * T) * payoff
    price = np.mean(discounted_payoff)
    std_error = np.std(discounted_payoff, ddof=1) / np.sqrt(n)
    return price, std_error


def antithetic_monte_carlo(S0, K, r, sigma, T, m, n, seed=None):
    """
    Use the antithetic variate method: Generate a pair of opposite random numbers for each path.
    Calculate the payoff of the lookback call option for both paths and then take the average.
    """
    if seed is not None:
        np.random.seed(seed)
    dt = T / m
    Z = np.random.randn(n, m)
    increments = (r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z
    increments_anti = (r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * (-Z)

    logS = np.log(S0) + np.cumsum(increments, axis=1)
    S = np.exp(logS)
    logS_anti = np.log(S0) + np.cumsum(increments_anti, axis=1)
    S_anti = np.exp(logS_anti)

    payoff = np.maximum(np.max(S, axis=1) - K, 0)
    payoff_anti = np.maximum(np.max(S_anti, axis=1) - K, 0)

    avg_payoff = 0.5 * (payoff + payoff_anti)
    discounted_avg_payoff = np.exp(-r * T) * avg_payoff
    price = np.mean(discounted_avg_payoff)
    std_error = np.std(discounted_avg_payoff, ddof=1) / np.sqrt(n)
    return price, std_error


def simulate_paths_both(S0, K, r, sigma, T, m, n, seed=None):
    """
    Generate paths simultaneously and return the payoff of the lookback call option and the payoff of the European call option
    """
    if seed is not None:
        np.random.seed(seed)
    dt = T / m
    Z = np.random.randn(n, m)
    increments = (r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z
    logS = np.log(S0) + np.cumsum(increments, axis=1)
    S = np.exp(logS)
    S_max = np.max(S, axis=1)
    lookback_payoff = np.maximum(S_max - K, 0)
    S_T = S[:, -1]
    european_payoff = np.maximum(S_T - K, 0)
    return lookback_payoff, european_payoff


def control_variate_monte_carlo(S0, K, r, sigma, T, m, n, seed=None):
    """
    Use the control variate method: Take the European call option as the control variate.
    """
    lookback_payoff, european_payoff = simulate_paths_both(
        S0, K, r, sigma, T, m, n, seed
    )
    discounted_lookback = np.exp(-r * T) * lookback_payoff
    discounted_european = np.exp(-r * T) * european_payoff
    european_call_theo = black_scholes_call(S0, K, T, r, sigma)

    # Calculate the sample covariance and variance (discounted values)
    cov = np.cov(discounted_lookback, discounted_european, ddof=1)[0, 1]
    var_european = np.var(discounted_european, ddof=1)
    b_star = cov / var_european

    # Estimator corrected by the control variate
    adjusted_payoff = discounted_lookback - b_star * (
        discounted_european - european_call_theo
    )
    price = np.mean(adjusted_payoff)
    std_error = np.std(adjusted_payoff, ddof=1) / np.sqrt(n)
    return price, std_error, b_star


# Parameter settings
S0 = 50
K = 55
r = 0.05
sigma = 0.2
T = 1
m = 50
n = 10000
seed = 42  # For reproducibility

# Plain Monte Carlo estimation
plain_price, plain_se = plain_monte_carlo(S0, K, r, sigma, T, m, n, seed)
# Antithetic Sampling estimation
anti_price, anti_se = antithetic_monte_carlo(S0, K, r, sigma, T, m, n, seed)
# Control Variate method estimation
control_price, control_se, b_star = control_variate_monte_carlo(
    S0, K, r, sigma, T, m, n, seed
)

print("Plain Monte Carlo: Price = {:.4f}, SE = {:.4f}".format(plain_price, plain_se))
print("Antithetic Sampling: Price = {:.4f}, SE = {:.4f}".format(anti_price, anti_se))
print(
    "Control Variate: Price = {:.4f}, SE = {:.4f}, b* = {:.4f}".format(
        control_price, control_se, b_star
    )
)

Plain Monte Carlo: Price = 4.9866, SE = 0.0672
Antithetic Sampling: Price = 5.0058, SE = 0.0343
Control Variate: Price = 5.0044, SE = 0.0283, b* = 1.0387
